# Monte-Carlo For Menzel 2015 Econometrica

In [1]:
using LinearAlgebra, Statistics, ForwardDiff, JuMP, LinearAlgebra, Distributions, Random
using Optim
using Optim: converged, maximum, maximizer, minimizer, iterations 

ArgumentError: ArgumentError: Package JuMP not found in current path:
- Run `import Pkg; Pkg.add("JuMP")` to install the JuMP package.


# The Following Function Calculates the systematic parts of the Pay-Offs

In [3]:
function makeindex(X_women,X_men,θ_w)
    nw,k=size(X_women)
    nm,k=size(X_men)
    
    tw_own=θ_w[1:k]
    tw_oth=θ_w[k+1:2*k]
    tw_diff=θ_w[2*k+1:3*k]
    tw_dist=θ_w[3*k+1:4*k]
    
    Xb = reshape((repeat(X_women,nm,1)-reshape(repeat(X_men',nw,1),k,nm*nw)').^2*tw_dist,nw,nm);
    
    Xb = Xb + repeat(X_women*tw_own,1,nm) + repeat(tw_oth',nw,1)*X_men';
    Xb = Xb + repeat(X_women*tw_diff,1,nm) - repeat(tw_diff',nw,1)*X_men';
    
    return Xb
end
    

makeindex (generic function with 1 method)

# The Gale-Shapley: Proposer = U 

In [4]:
function Gale_Shapley(U,V)
    nw,nm=size(U)
    U_temp=U
    nmax=10*nw*nm
    
    for i in 1:nmax
        Prop = Int.((U_temp.==repeat(max.(findmax(U_temp,dims=2)[1],0),1,nm)))
        Rej = Prop.*(Int.((Prop.*V .< max.(0,repeat(findmax(Prop.*V,dims=1)[1],nw,1)))))
        U_temp[Rej.== Prop].=-1;
        
        if sum(sum(Rej))==0
            break
        end
        
    end
    
   return μ= Int.((U_temp.==repeat(max.(findmax(U_temp,dims=2)[1],0),1,nm)))
    
end
    

Gale_Shapley (generic function with 1 method)

# The Loglikelihood Function

In [230]:
function loglik2(θ::Vector,μ::AbstractArray,X::AbstractArray,Z::AbstractArray)
    
    for i in 16:20
        if θ[i]<=0
        θ[i]=1e-4
        end
    end
    
   # θ[3:7]=zeros(5)
   # θ[11:15]=zeros(5)
        
    nw=length(X);
    nm=length(Z);
    k=Int(length(θ)/2-2);
    
    βw=θ[1:k];
    βm=θ[k+1:2*k];
    Γ=θ[2*k+1:2*k+4]
    
    if  Γ <= zeros(4)
        Γ.= 1e-2
    end

    
    Γ_w_mat = log.((1 .+ Γ[1].*(1 .-X[:,2]) + Γ[2].*X[:,2]));
    Γ_m_mat = log.((1 .+ Γ[3].*(1 .-Z[:,2]) + Γ[4].*Z[:,2]));
    
    U_star = makeindex(X,Z,βw);
    V_star = makeindex(Z,X,βm)';
   
    W_star = μ.*(U_star + V_star);
    
    
    mar_w = sum(μ,dims=2);
    mar_m = sum(μ,dims=1)';
    
    LL = -(2*sum(sum(W_star)) - sum((1 .+mar_w).*Γ_w_mat) - sum((1 .+mar_m).*Γ_m_mat));
    return LL
    
end

 loglik2(-1.5 .*ones(20),μ,X,Z)
#theta_0

201.00419970001997

# Fixed Point Constraint

In [229]:
function Ψ_fp1(θ::Vector,X::AbstractArray,Z::AbstractArray)
    
    for i in 16:20
        if θ[i]<=0
        θ[i]=1e-4
        end
    end
      
   # θ[3:7]=zeros(5)
    #θ[11:15]=zeros(5)
    
    
    nw = length(X);
    nm = length(Z);
    k = Int(length(θ)/2-2);
    β_w = θ[1:k];
    β_m = θ[k+1:2*k];
    Γ = θ[2*k+1:2*k+4];
    
    
    Γ_w_mat = 1 .+ Γ[1].*(1 .-X[:,2]) + Γ[2].*X[:,2];
    Γ_m_mat = 1 .+ Γ[3].*(1 .-Z[:,2]) + Γ[4].*Z[:,2];
    
    Xw = [1 0;1 1];
    
    U_star = makeindex(Xw,Z,β_w);
    V_star = makeindex(Z,Xw,β_m)';
    W_star_w = U_star + V_star;
    
    Zm = [1 0;1 1];
    
    
    U_star = makeindex(X,Zm,β_w);
    V_star = makeindex(Zm,X,β_m)';
    W_star_m = U_star + V_star;
    
    Ψ_w = mean((exp.(W_star_w))' ./repeat(Γ_m_mat,1,2),dims=1);
    Ψ_m = mean(exp.(W_star_m)./repeat(Γ_w_mat,1,2),dims=1);
    Ψ_diff1 = Γ - [Ψ_w'..., Ψ_m'...]
    
    #Ψ_ineq = zeros(4,1);
    return Ψ_diff1
    
end

Ψ_fp1 (generic function with 2 methods)

# Parametric Consistency Constraint

In [182]:
function θ_diff(θ)
    
    th_diff=θ[1:8]-θ[9:16]
    return th_diff
end
    

θ_diff (generic function with 1 method)

# Lagrange For The Optimization

In [269]:
function lagrangian(θλ::Vector,μ::AbstractArray,X::AbstractArray,Z::AbstractArray)
    k=8
    θ=θλ[1:2*k+4]
    λ1=θλ[2*k+5:2*k+8]
    λ2=θλ[3*k+1:4*k]
    λ3=θλ[33:42]
    λ4=θλ[43:52]
    lagr=loglik2(θ,μ,X,Z)+dot(λ1,Ψ_fp1(θ,X,Z))+dot(λ2,θ_diff(θ))+dot(λ3,[θ[3:7]...,θ[11:15]...])-dot(λ4,[θ[3:7]...,θ[11:15]...])
    return lagr
end

function lagprime(θλ::Vector,μ::AbstractArray,X::AbstractArray,Z::AbstractArray)
     lag=y->ForwardDiff.gradient(y->lagrangian(y,μ,X,Z),y)
    return lag(θλ)
end

lagprime (generic function with 1 method)

# Define Loglikelihood Function to accomodate NLopt Format

In [6]:
function lognlopt!(θ::Vector,grad::Vector,μ::AbstractArray,X::AbstractArray,Z::AbstractArray)
    logtmp(x)=loglik2(x,μ,X,Z);  
    tmp= y->ForwardDiff.gradient(logtmp,y)
    if length(grad)>0
        grad= tmp(θ)
    end
    
    return logtmp(θ)
end
    

lognlopt (generic function with 1 method)

# Define Fixed Point Constraint to accomodate NLopt Format

In [7]:
function Ψ_nlopt!(result::Vector,θ::Vector,grad::Array,X::AbstractArray,Z::AbstractArray)
    Ψ_tmp(q) =Ψ_fp1(q,X,Z);
    tmp= y->ForwardDiff.jacobian(Ψ_tmp,y)    
    if length(grad)>0
        grad=tmp(θ)
    end
    return [Ψ_tmp(θ)...]
end

Ψ_nlopt (generic function with 1 method)

# MonteCarlo

In [251]:
        
# Random.seed!(1234)

NumberofSimulations = 20

beta_sim=zeros(52,NumberofSimulations)

for b in 1:NumberofSimulations

        n=20
        alpha=0.5
        n2 =Int(n/2)
        X = repeat([1.0 0.0;1.0 1.0],n2,1)
        Z = repeat([1.0 0.0;1.0 1.0],n2,1)

        beta_w = [1;0;0;0;0;0;0;1];
        beta_m = [1;0;0;0;0;0;0;1];
        
        beta_w = [alpha;0.5;0;0;0;0;0;1];
        beta_m = [alpha;0.5;0;0;0;0;0;1];
       
        
        U_star = makeindex(X,Z,beta_w);
        V_star = makeindex(Z,X,beta_m)'

        η=-log.(-log.(rand(n,n)));
        ζ=-log.(-log.(rand(n,n)))
        
        J=Int(round(sqrt(n)))
        η₀=repeat(findmax(-log.(-log.(rand(J,n))),dims=1)[1]',1,n)
        ζ₀=repeat(findmax(-log.(-log.(rand(J,n))),dims=1)[1],n,1)

        η=η-η₀
        ζ=ζ-ζ₀
        
        U=U_star+η
        V=V_star+ζ

        μ=Gale_Shapley(U,V)
        

        k = length(beta_w);
        
       #theta_0 = [beta_w...,beta_m...,zeros(4)...];
        theta_0 = [0.5 .*ones(20)...]

        
        thetalambda_0=[theta_0...,0.5 .*ones(32)...]
        
        
        lower = [zeros(52)...]
        upper= [1,1,ones(5)...,1,1,1,ones(5)...,1,1 .*ones(36)...]
        
        results = Optim.optimize(y->lagrangian(y,μ,X,Z),lower,upper, thetalambda_0,Fminbox() )
        println("minimum = $(results.minimum) with argmin = $(results.minimizer[1:8]) in $(results.iterations) iterations")  beta_sim[:,b]=results.minimizer
end  
@show beta_sim
a=mean(beta_sim[1:k,:],dims=2)       

ArgumentError: ArgumentError: Value and slope at step length = 0 must be finite.